In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt

In [2]:
#Simulo datos
np.random.seed(42)
n = 20000

In [3]:
eps1 = np.random.normal(0, 1, n)
eps2 = np.random.normal(0, 1, n)
eps3 = np.random.normal(0, 1, n)
epsx = np.random.normal(0, 1, n)
epsy = np.random.normal(0, 1, n)

In [4]:
Z1 = eps1
Z3 = eps3
Z2 = Z3 + eps2
X = Z1 + Z2 + epsx
Y = X + Z1 + Z2 + Z3 + epsy

In [5]:
df = pd.DataFrame({'Y': Y, 'X': X, 'Z1': Z1, 'Z2': Z2, 'Z3': Z3})

In [6]:
def run_ols(vars_x):
    Xmat = sm.add_constant(df[vars_x])
    model = sm.OLS(df['Y'], Xmat).fit()
    beta = model.params['X']
    se = model.bse['X']
    z = 2.5758  # IC al 99%
    ci_low, ci_high = beta - z * se, beta + z * se
    return model, beta, se, (ci_low, ci_high)

In [7]:
modelos = [
    ["X"],
    ["X", "Z1"],
    ["X", "Z2"],
    ["X", "Z1", "Z2"],
    ["X", "Z1", "Z2", "Z3"]
]

In [8]:
results = []
for m in modelos:
    res, b, se, ci = run_ols(m)
    results.append({"Vars": m, "Beta_X": b, "SE": se, "CI_low": ci[0], "CI_high": ci[1], "Model": res})

In [10]:
tabla = pd.DataFrame(results)
tabla["Regresión"] = ["Y~X", "Y~X+Z1", "Y~X+Z2", "Y~X+Z1+Z2", "Y~X+Z1+Z2+Z3"]
tabla = tabla[["Regresión", "Beta_X", "SE", "CI_low", "CI_high"]]
print("RESULTADOS DE LAS REGRESIONES (IC 99%)")
print(tabla.round(4))

In [12]:
plt.figure(figsize=(8,5))
xpos = np.arange(len(tabla))
plt.errorbar(xpos, tabla["Beta_X"],
             yerr=[tabla["Beta_X"] - tabla["CI_low"], tabla["CI_high"] - tabla["Beta_X"]],
             fmt='o', color='orange', capsize=5)
plt.axhline(1, color='black', linestyle='--', label='Valor verdadero = 1')
plt.xticks(xpos, tabla["Regresión"])
plt.ylabel("Estimador de β_X")
plt.title("Estimaciones del efecto de X sobre Y con IC 99%")
plt.legend()
plt.tight_layout()
plt.savefig("Estimaciones.png")
plt.show()

El gráfico muestra las estimaciones del efecto de X sobre Y bajo distintas especificaciones de control. Se observa que los modelos (1) Y~X, (2) Y~X+Z1 y (3) Y~X+Z2 sobreestiman el efecto verdadero (β=1), indicando sesgo por variables omitidas, ya que al no controlar simultáneamente por Z1 y Z2, los caminos de confusión permanecen abiertos. En cambio, las regresiones (4) Y~X+Z1+Z2 y (5) Y~X+Z1+Z2+Z3 producen estimaciones cercanas al valor real, confirmando que incluir Z1 y Z2 es suficiente para identificar correctamente el efecto causal. Además, al añadir Z3 no se mejora la precisión ni el sesgo, sino que se amplía ligeramente el intervalo de confianza debido a multicolinealidad innecesaria. En síntesis, controlar por Z1 y Z2 basta para obtener una estimación insesgada y eficiente del efecto de X sobre Y.